In [ ]:
import triangle
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from math import ceil, sqrt
from scipy.sparse import csr_matrix

import finis

%matplotlib notebook

### Triangulation for Testing

In [ ]:
g = {
    'vertices': np.copy(np.array([[0, 3, 3, 0], [0, 0, 1.5, 1.5]]).T),
    'segments': np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
}
maxArea = 3
mesh = triangle.triangulate(g,"pDa"+str(maxArea))
integ = finis.build_integ(mesh, order=6)
plt.figure()
finis.plot_mesh(mesh, vertex_numbers=True, triangle_numbers=True)
plt.axis("equal")
plt.show()

### Question 1 & 2

In [ ]:
%%latex
\begin{align}
\mathbb{P}_0: &&  \phi_0(x,y) &\equiv 1 \\[2ex]
%
\mathbb{P}_1:   &&  \phi_0(x,y) &= 1 - x - y \\
                &&  \phi_1(x,y) &= x \\
                &&  \phi_2(x,y) &= y
\end{align}

In [ ]:
def lagrange_1d(x_in, order=1):
    N = x_in.size
    
    x = x_in
    
    if len(x_in.shape) == 1:
        x = x_in[:, None]
    
    if order == 0:
        return np.ones((N, 1)), np.zeros((N, 1))
    
    if order == 1:
        phi = np.hstack((1.0-x, x))
        dxphi = np.hstack((-np.ones((N, 1)), np.ones((N, 1))))
        
        assert(phi.flags['OWNDATA'])
        assert(dxphi.flags['OWNDATA'])
        
        return  phi, dxphi
        
    raise NotImplementedError("Only orders 0 and 1 are implemented")

    
def lagrange_2d(x_in, y_in, order=1):
    N = x_in.size
    
    x = x_in
    y = y_in
    
    if len(x_in.shape) == 1:
        x = x_in[:, None]
    if len(y_in.shape) == 1:
        y = y_in[:, None]
    
    if order == 0:
        return np.ones((N, 1)), np.zeros((N, 1)), np.zeros((N, 1))
    
    if order == 1:
        phi = np.hstack((1.0-x-y, x, y))
        dxphi = np.hstack((-np.ones((N, 1)), np.ones((N, 1)), np.zeros((N, 1))))
        dyphi = np.hstack((-np.ones((N, 1)), np.zeros((N, 1)), np.ones((N, 1))))
        
        assert(phi.flags['OWNDATA'])
        assert(dxphi.flags['OWNDATA'])
        assert(dyphi.flags['OWNDATA'])
        
        return  phi, dxphi, dyphi
        
    raise NotImplementedError("Only orders 0 and 1 are implemented")

### Question 3 & 4

In [ ]:
def fe_space(mesh, fe_type = 'lagrange', order = 1, formula = finis.form_int_2d):
    """fe = finite element"""
    
    if fe_type.lower() != 'lagrange':
        raise NotImplementedError("Only Lagrange elements implemented")
        
    if (order < 0) or (order > 1):
        raise NotImplementedError("Only orders 0 and 1 are implemented")
    
    if order == 0:
        N_tri = mesh['triangles'].shape[0] # Number of triangles
        N_int = 3 # number integration points per triangle
        N_b = 1 # number of dof per triangle
                
        rows = np.arange(N_tri*N_int*N_b)
        cols = np.repeat(np.arange(N_tri), N_int)
        data = np.ones((N_tri*N_int*N_b,), dtype=np.float)
                    
        fe = {
            'num_dof': np.arange(N_tri)[:,None],
            'dof': finis.triangle_cog(mesh),
            'U': csr_matrix((data, (rows, cols)), shape=(N_tri*N_int, N_tri*N_b))
        }
        
    if order == 1:
        N_tri = mesh['triangles'].shape[0] # Number of triangles
        N_int = 3 # number integration points per triangle
        N_b = 3 # number of dof per triangle
        
        xh, yh, wh = formula()
        phi, dxphi, dyphi = lagrange_2d(xh, yh, order=order)
        
        # Attention: The following code is fully vectorized and horrible to understand. Sorry
        
        ### U ###
        rows = np.repeat(np.arange(N_tri*N_int), N_b)
        cols = np.tile(mesh['triangles'], reps=(1,N_int)).flatten(order='C')
        data = np.tile(phi.flatten(order='C'), N_tri)
        
        U = csr_matrix((data, (rows, cols)), shape=(N_tri*N_int, mesh['vertices'].shape[0]))
        
        
        ### Derivative Transforms ###
        xT = finis.triangle_x(mesh)
        yT = finis.triangle_y(mesh)
    
        dx_dxh = xT[:,1]-xT[:,0]
        dx_dyh = xT[:,2]-xT[:,0]
        dy_dxh = yT[:,1]-yT[:,0]
        dy_dyh = yT[:,2]-yT[:,0]
        
        det = dx_dxh*dy_dyh - dx_dyh*dy_dxh
        det_inv = 1.0 / det
        assert np.amax(np.abs(det_inv)) < 1e5, "Badly scaled triangles"
        
        dxh_dx = det_inv * dy_dyh
        dyh_dy = det_inv * dx_dxh
        dxh_dy = - det_inv * dx_dyh
        dyh_dx = - det_inv * dy_dxh
        
        
        ### Transform Differentials ###
        dphi_dxh = np.tile(dxphi.flatten(order='C'), N_tri)
        dphi_dyh = np.tile(dyphi.flatten(order='C'), N_tri)
        
        dphi_dx = dphi_dxh * np.repeat(dxh_dx, N_b*N_int) + dphi_dyh * np.repeat(dyh_dx, N_b*N_int)
        dphi_dy = dphi_dxh * np.repeat(dxh_dy, N_b*N_int) + dphi_dyh * np.repeat(dyh_dy, N_b*N_int)
        
        
        ### DUX, DUY ###
        DUX = csr_matrix((dphi_dx, (rows, cols)), shape=(N_tri*N_int, mesh['vertices'].shape[0]))
        DUY = csr_matrix((dphi_dy, (rows, cols)), shape=(N_tri*N_int, mesh['vertices'].shape[0]))
        
        fe = {
            'num_dof': np.copy(mesh['triangles']),
            'dof': np.copy(mesh['vertices']),
            'U': U,
            'DUX': DUX,
            'DUY': DUY,
            'w': np.repeat(np.abs(det), N_int) * np.tile(wh, N_tri)
        }
        
    return fe

In [ ]:
order = 1
fe = fe_space(mesh)
integ = finis.build_integ(mesh)

plt.figure()
finis.plot_mesh(mesh, vertex_numbers=True, triangle_numbers=True)
plt.plot(fe['dof'][:,0], fe['dof'][:,1], 'ko', label="DOF, k = {}".format(order))
plt.plot(integ['x'], integ['y'], '+r', label='integ')
for i in range(len(integ['x'])):
    plt.text(integ['x'][i], integ['y'][i], str(i), color='r')
plt.legend()
plt.axis("equal")
plt.show()

print("U")
print(fe['U'].toarray())
print("DUX")
print(fe['DUX'].toarray())
print("DUY")
print(fe['DUY'].toarray())

### Question 6

In [ ]:
g = {
    'vertices': np.copy(np.array([[0, 1, 1, 0], [0, 0, 1., 1.]]).T),
    'segments': np.copy(np.array([[0, 1, 2, 3], [1, 2, 3, 0]]).T)
}
maxArea = 0.001
mesh = triangle.triangulate(g,"qpDa"+str(maxArea))

print("number of vertices = {}".format(mesh['vertices'].shape[0]))

In [ ]:
plt.figure()
finis.plot_mesh(mesh, vertex_numbers=True, triangle_numbers=True)
plt.plot(fe['dof'][:,0], fe['dof'][:,1], 'ko', label="DOF, k = {}".format(order))
plt.plot(integ['x'], integ['y'], '+r', label='integ')
for i in range(len(integ['x'])):
    plt.text(integ['x'][i], integ['y'][i], str(i), color='r')
plt.legend()
plt.axis("equal")
plt.show()

In [ ]:
fun = lambda x, y: np.exp(x)
dx_fun = lambda x, y: np.exp(x)
dy_fun = lambda x, y: np.zeros_like(x)

%time fe = fe_space(mesh, order=1)

u_h = fun(mesh['vertices'][:,0], mesh['vertices'][:,0])
u_int = fe['U'].dot(u_h)
dx_u_int = fe['DUX'].dot(u_h)
dy_u_int = fe['DUY'].dot(u_h)


integ = finis.build_integ(mesh)
u_int_control = fun(integ['x'], integ['y'])
dx_u_int_control = dx_fun(integ['x'], integ['y'])
dy_u_int_control = dy_fun(integ['x'], integ['y'])


print("Max err. at x_int (eval) = {}".format(np.amax(np.abs(u_int - u_int_control))))
print("Max err. at x_int (d/dx) = {}".format(np.amax(np.abs(dx_u_int - dx_u_int_control))))
print("Max err. at x_int (d/dy) = {}".format(np.amax(np.abs(dy_u_int - dy_u_int_control))))

print("\nNumeric Integral  = {}".format(np.sum(u_int*fe['w'])))
print("Analytic Integral = {}".format(np.expm1(1)))

print("Number of vertices   = {}".format(u_h.size))
print("Number of int points = {}".format(u_int.size))

In [ ]:
fun_f = lambda x, y: np.exp(x+y)
fun_g = lambda x, y: np.sin(x**3 - y**5)

%time fe = fe_space(mesh, order=1)

u_h_f = fun_f(mesh['vertices'][:,0], mesh['vertices'][:,0])
u_h_g = fun_g(mesh['vertices'][:,0], mesh['vertices'][:,0])

u_int_f = fe['U'].dot(u_h_f)
u_int_g = fe['U'].dot(u_h_g)

dx_u_int_f = fe['DUX'].dot(u_h_f)
dx_u_int_g = fe['DUX'].dot(u_h_g)

I1 = np.sum(fe['w'] * u_int_f * dx_u_int_g)
I2 = np.sum(fe['w'] * u_int_g * dx_u_int_f)

print(I1+I2)